In [ ]:
!pip install gdown
!gdown --id 1XQoAg_iIXkc2rIhdU2LMQSLVKWoYhYJl
!unzip vectorized_outputs.zip -d vectorized_outputs

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1XQoAg_iIXkc2rIhdU2LMQSLVKWoYhYJl
From (redirected): https://drive.google.com/uc?id=1XQoAg_iIXkc2rIhdU2LMQSLVKWoYhYJl&confirm=t&uuid=bea1ddd3-c9d2-4ad5-a382-57d10b32ed87
To: /content/vectorized_outputs.zip
100% 2.32G/2.32G [00:31<00:00, 74.3MB/s]
Archive:  vectorized_outputs.zip
  inflating: vectorized_outputs/vectorized_outputs.pt  


In [ ]:
import torch
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import random

In [ ]:
file_dir = '/content/vectorized_outputs/vectorized_outputs.pt'
ds_tensor = torch.load(file_dir)
probing_set = pd.read_csv("https://drive.google.com/uc?id=1UimnWZypzwkMtLSor7qv2q9CJP6GCuTa")
probing_set.head(5)

,Unnamed: 0.1,Unnamed: 0,base,polypers,was_changed,normal,distortion
0,3,82,Путник вынул хлебец из сумки и положил на стол .,Путник вынулет хлебец из сумки и положил на ст...,True,True,Путник вынул хлебец из сумки и положил на стол .
1,4,83,Хозяин взял хлебец в руку и долго смотрел на у...,Хозяин взялет хлебец в руку и долго смотрел на...,True,True,Хозяин взял хлебец в руку и долго смотрел на у...
2,7,153,"Мы работаем на них , а они забирают рис у наши...","Мы работаем на них , а они забираютет рис у на...",True,True,"Мы работаем на них , а они забирают рис у наши..."
3,8,169,"Мы прогоним их обратно в море , из которого он...","Мы прогонимете их обратно в море , из которого...",True,True,"Мы прогоним их обратно в море , из которого он..."
4,9,193,Райот разбивал мотыгой сухую каменистую землю .,Райот разбивалет мотыгой сухую каменистую землю .,True,True,Райот разбивал мотыгой сухую каменистую землю .


In [ ]:
rubert_vec_base, rubert_vec_polypers, rubert_vec_distor = ds_tensor['rubert'].unbind(dim=1)
vec_base, vec_polypers, vec_distor = ds_tensor['custom'].unbind(dim=1)

In [ ]:
probing_set['vec_distor'] = [vec.numpy() for vec in vec_distor]
probing_set['vec_polypers'] = [vec.numpy() for vec in vec_polypers]
probing_set['vec_base'] = [vec.numpy() for vec in vec_base]

probing_set['rubert_vec_base'] = [vec.numpy() for vec in rubert_vec_base]
probing_set['rubert_vec_polypers'] = [vec.numpy() for vec in rubert_vec_polypers]
probing_set['rubert_vec_distor'] = [vec.numpy() for vec in rubert_vec_distor]

# Недоделанное

In [ ]:
X = np.stack(probing_set['vec_distor'].values)
y = probing_set['normal'].values
print(X.shape, y.shape)

(229849, 768) (229849,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [ ]:
preds = logreg.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

       False       0.99      0.99      0.99     11543
        True       0.99      0.99      0.99     11442

    accuracy                           0.99     22985
   macro avg       0.99      0.99      0.99     22985
weighted avg       0.99      0.99      0.99     22985



In [ ]:
abnormal_set = probing_set[probing_set['normal'] == False].sample(n=59801, random_state=42)
filtered_set = pd.concat([probing_set[probing_set.was_changed == True], abnormal_set])

In [ ]:
X = np.stack(filtered_set['vec_distor'].values)
y = filtered_set['was_changed'].values
print(X.shape, y.shape)

(119602, 768) (119602,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

logreg_pol = LogisticRegression(max_iter=1000)
logreg_pol.fit(X_train, y_train)

preds = logreg_pol.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

       False       0.99      0.99      0.99      5817
        True       0.99      0.99      0.99      6144

    accuracy                           0.99     11961
   macro avg       0.99      0.99      0.99     11961
weighted avg       0.99      0.99      0.99     11961



In [ ]:
probl_c = 0
for i in range(len(vec_polypers)):
    if np.array_equal(vec_polypers[i], vec_distor[i]) != True:
        probl_c += 1
probl_c

174658

In [ ]:
probing_set[probing_set.polypers == probing_set.distortion]

,Unnamed: 0.1,Unnamed: 0,base,polypers,was_changed,normal,distortion,vec_distor,vec_polypers
59801,90686,906581,"Совсем дети , а любовь к Родине , ненависть к ...","Совсем дети , а любовь к Родине , ненависть к ...",False,True,"Совсем дети , а любовь к Родине , ненависть к ...","[-0.1661889, -0.073595464, 0.016332349, 0.0720...","[-0.17037068, -0.06750146, 0.016696705, 0.0711..."
59802,217573,2176722,Короткое время спустя стройная фигура Сирайо с...,Короткое время спустя стройная фигура Сирайо с...,False,True,Короткое время спустя стройная фигура Сирайо с...,"[-0.2192254, -0.17074871, 0.121937625, 0.03948...","[-0.22024606, -0.16977052, 0.12159621, 0.03914..."
59803,153532,1537467,"Хвост остается висеть в небе , он постепенно г...","Хвост остается висеть в небе , он постепенно г...",False,True,"Хвост остается висеть в небе , он постепенно г...","[-0.24896835, -0.19558243, 0.09573273, 0.00823...","[-0.24956605, -0.19546421, 0.09554559, 0.00826..."
59804,105826,1057221,— Нам не терпится скорее познакомиться с Атилл...,— Нам не терпится скорее познакомиться с Атилл...,False,True,— Нам не терпится скорее познакомиться с Атилл...,"[-0.08965833, -0.06801912, 0.07732145, 0.07526...","[-0.08965833, -0.06801912, 0.07732145, 0.07526..."
59805,177808,1779963,В машине молчали .,В машине молчали .,False,True,В машине молчали .,"[-0.11885286, -0.1679682, 0.013121893, 0.05724...","[-0.11885286, -0.1679682, 0.013121893, 0.05724..."
...,...,...,...,...,...,...,...,...,...
227507,177034,1772044,Туда-обратно .,Туда-обратно .,False,False,Туда-обратно .,"[-0.19153325, -0.16988309, 0.107153095, 0.0679...","[-0.18675935, -0.17132728, 0.10378289, 0.06977..."
227946,129286,1293916,Га-ля .,Га-ля .,False,False,Га-ля .,"[-0.14769015, -0.1299483, 0.13349567, 0.042855...","[-0.14204876, -0.13503475, 0.13232987, 0.04411..."
227986,4743,47230,[ 4 ] .,[ 4 ] .,False,False,[ 4 ] .,"[-0.18075585, -0.15361176, 0.14652367, 0.06368...","[-0.18453324, -0.15455143, 0.14411871, 0.06170..."
228497,124892,1249917,— Вот-вот …,— Вот-вот …,False,False,— Вот-вот …,"[-0.18315479, -0.084761575, 0.05590874, 0.1222...","[-0.17415221, -0.08112613, 0.062411554, 0.1221..."


In [ ]:
probing_set[probing_set.base == probing_set.distortion]

,Unnamed: 0.1,Unnamed: 0,base,polypers,was_changed,normal,distortion,vec_distor,vec_polypers
0,3,82,Путник вынул хлебец из сумки и положил на стол .,Путник вынулет хлебец из сумки и положил на ст...,True,True,Путник вынул хлебец из сумки и положил на стол .,"[-0.124079876, -0.1180061, -0.0045949635, 0.05...","[-0.12234213, -0.116294704, 0.0007983434, 0.06..."
1,4,83,Хозяин взял хлебец в руку и долго смотрел на у...,Хозяин взялет хлебец в руку и долго смотрел на...,True,True,Хозяин взял хлебец в руку и долго смотрел на у...,"[-0.17452149, -0.17705852, 0.03562201, 0.03482...","[-0.18793519, -0.1718917, 0.02643905, 0.028724..."
2,7,153,"Мы работаем на них , а они забирают рис у наши...","Мы работаем на них , а они забираютет рис у на...",True,True,"Мы работаем на них , а они забирают рис у наши...","[-0.16748263, -0.17300434, 0.1150845, 0.032609...","[-0.17429209, -0.16757962, 0.11316887, 0.03448..."
3,8,169,"Мы прогоним их обратно в море , из которого он...","Мы прогонимете их обратно в море , из которого...",True,True,"Мы прогоним их обратно в море , из которого он...","[-0.19267018, -0.16964169, 0.1373793, 0.101200...","[-0.19034386, -0.16327468, 0.13815689, 0.09367..."
4,9,193,Райот разбивал мотыгой сухую каменистую землю .,Райот разбивалет мотыгой сухую каменистую землю .,True,True,Райот разбивал мотыгой сухую каменистую землю .,"[-0.11212488, -0.1295488, -0.016064871, 0.0051...","[-0.12360215, -0.12732175, -0.024102904, 0.023..."
...,...,...,...,...,...,...,...,...,...
227507,177034,1772044,Туда-обратно .,Туда-обратно .,False,False,Туда-обратно .,"[-0.19153325, -0.16988309, 0.107153095, 0.0679...","[-0.18675935, -0.17132728, 0.10378289, 0.06977..."
227946,129286,1293916,Га-ля .,Га-ля .,False,False,Га-ля .,"[-0.14769015, -0.1299483, 0.13349567, 0.042855...","[-0.14204876, -0.13503475, 0.13232987, 0.04411..."
227986,4743,47230,[ 4 ] .,[ 4 ] .,False,False,[ 4 ] .,"[-0.18075585, -0.15361176, 0.14652367, 0.06368...","[-0.18453324, -0.15455143, 0.14411871, 0.06170..."
228497,124892,1249917,— Вот-вот …,— Вот-вот …,False,False,— Вот-вот …,"[-0.18315479, -0.084761575, 0.05590874, 0.1222...","[-0.17415221, -0.08112613, 0.062411554, 0.1221..."


# Классификация на мЕньшей выборке

In [ ]:
poly = probing_set[probing_set.was_changed == True]
poly = poly[['vec_polypers', 'rubert_vec_polypers']]

In [ ]:
normalset = probing_set[probing_set.was_changed == False]
normalset = normalset[normalset.normal == True]
normalset = normalset[['vec_polypers', 'rubert_vec_polypers']]

In [ ]:
correct_sent = pd.concat([poly, normalset])
len(correct_sent)

114924

In [ ]:
abnormal_set = probing_set[probing_set.was_changed == False]
abnormal_set = abnormal_set[abnormal_set.normal == False]

In [ ]:
print(len(abnormal_set[abnormal_set.distortion != abnormal_set.base]),
      len(abnormal_set[abnormal_set.apply(lambda row: np.array_equal(row['vec_base'],
                                                                     row['vec_distor']) == False,
                                          axis=1)]))

114682 114865


In [ ]:
mask = probing_set.apply(
    lambda row: (not np.array_equal(row['vec_base'], row['vec_distor']))
                and (row['base'] != row['distortion']),
    axis=1
)
abnormal_set = probing_set[mask].copy()
abnormal_sent = abnormal_set[['vec_distor', 'rubert_vec_distor']]

In [ ]:
correct_sent_sample = correct_sent.sample(len(abnormal_set), random_state=88)

correct_sent_sample['correct'] = 1
abnormal_sent['correct'] = 0
probe_set = pd.concat([abnormal_sent, correct_sent_sample])

<ipython-input-114-200110c3b3e2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abnormal_sent['correct'] = 0


In [ ]:
probe_set['combined_vec'] = np.where(probe_set['vec_distor'].notna(),
                                     probe_set['vec_distor'],
                                     probe_set['vec_polypers'])
probe_set['combined_rubert_vec'] = np.where(probe_set['rubert_vec_distor'].notna(),
                                    probe_set['rubert_vec_distor'],
                                    probe_set['rubert_vec_polypers'])
probe_set

,vec_distor,rubert_vec_distor,correct,vec_polypers,rubert_vec_polypers,combined_vec,combined_rubert_vec
114924,"[-0.08197381, -0.0480008, 0.030531961, 0.05012...","[0.17642552, 0.002420795, 0.42952782, 0.021984...",0,NaN,NaN,"[-0.08197381, -0.0480008, 0.030531961, 0.05012...","[0.17642552, 0.002420795, 0.42952782, 0.021984..."
114925,"[-0.14759514, -0.08897676, 0.021126665, 0.0496...","[-0.44947323, 0.04316721, 0.56528175, -0.15134...",0,NaN,NaN,"[-0.14759514, -0.08897676, 0.021126665, 0.0496...","[-0.44947323, 0.04316721, 0.56528175, -0.15134..."
114926,"[-0.17337358, -0.15158856, 0.10543449, 0.02940...","[-0.3656044, -0.22188976, -0.126323, -0.147110...",0,NaN,NaN,"[-0.17337358, -0.15158856, 0.10543449, 0.02940...","[-0.3656044, -0.22188976, -0.126323, -0.147110..."
114927,"[-0.19965608, -0.13574971, 0.10322923, 0.08905...","[-0.20491242, 0.11024289, 0.18544051, -0.05202...",0,NaN,NaN,"[-0.19965608, -0.13574971, 0.10322923, 0.08905...","[-0.20491242, 0.11024289, 0.18544051, -0.05202..."
114928,"[-0.088135965, -0.08134083, 0.013833351, 0.099...","[-0.08139154, -0.01260149, 0.20741166, 0.22745...",0,NaN,NaN,"[-0.088135965, -0.08134083, 0.013833351, 0.099...","[-0.08139154, -0.01260149, 0.20741166, 0.22745..."
...,...,...,...,...,...,...,...
98368,NaN,NaN,1,"[-0.03531903, -0.054362018, -0.001451126, 0.04...","[0.09415641, -0.16024196, 0.41410866, -0.20934...","[-0.03531903, -0.054362018, -0.001451126, 0.04...","[0.09415641, -0.16024196, 0.41410866, -0.20934..."
62676,NaN,NaN,1,"[-0.08907429, -0.024714088, 0.063186966, 0.074...","[0.34130588, 0.0097893225, 0.1538143, 0.202169...","[-0.08907429, -0.024714088, 0.063186966, 0.074...","[0.34130588, 0.0097893225, 0.1538143, 0.202169..."
33122,NaN,NaN,1,"[-0.18602358, -0.100935936, 0.11400763, 0.0649...","[0.26269504, 0.08344227, 0.37109715, -0.041011...","[-0.18602358, -0.100935936, 0.11400763, 0.0649...","[0.26269504, 0.08344227, 0.37109715, -0.041011..."
95212,NaN,NaN,1,"[-0.10388061, -0.09904459, 0.008202038, 0.0424...","[0.080757424, 0.09594997, 0.48750055, 0.078050...","[-0.10388061, -0.09904459, 0.008202038, 0.0424...","[0.080757424, 0.09594997, 0.48750055, 0.078050..."


In [ ]:
# с локализакией
X = np.stack(probe_set.combined_vec.values)
y = np.stack(probe_set.correct.values)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

logreg_pol = LogisticRegression(max_iter=1000)
logreg_pol.fit(X_train, y_train)

preds = logreg_pol.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98     11351
           1       0.98      0.99      0.98     11586

    accuracy                           0.98     22937
   macro avg       0.98      0.98      0.98     22937
weighted avg       0.98      0.98      0.98     22937



In [ ]:
# без локализации
X = np.stack(probe_set.combined_rubert_vec.values)
y = np.stack(probe_set.correct.values)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

logreg_pol = LogisticRegression(max_iter=1000)
logreg_pol.fit(X_train, y_train)

preds = logreg_pol.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99     11351
           1       0.99      0.99      0.99     11586

    accuracy                           0.99     22937
   macro avg       0.99      0.99      0.99     22937
weighted avg       0.99      0.99      0.99     22937



In [ ]:
abnor2 = abnormal_sent.sample(len(poly), random_state=80)
abnor2['poly'] = 0
poly['poly'] = 1
new_set = pd.concat([abnor2, poly])
new_set['combined_vec'] = np.where(new_set['vec_distor'].notna(),
                                     new_set['vec_distor'],
                                     new_set['vec_polypers'])
new_set['combined_rubert_vec'] = np.where(new_set['rubert_vec_distor'].notna(),
                                    new_set['rubert_vec_distor'],
                                    new_set['rubert_vec_polypers'])
len(new_set)/2

59801.0

In [ ]:
# с локализакией
X = np.stack(new_set.combined_vec.values)
y = np.stack(new_set.poly.values)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

logreg_pol = LogisticRegression(max_iter=1000)
logreg_pol.fit(X_train, y_train)

preds = logreg_pol.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98      6144
           1       0.98      0.99      0.98      5817

    accuracy                           0.98     11961
   macro avg       0.98      0.98      0.98     11961
weighted avg       0.98      0.98      0.98     11961



In [ ]:
# без локализации
X = np.stack(new_set.combined_rubert_vec.values)
y = np.stack(new_set.poly.values)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

logreg_pol = LogisticRegression(max_iter=1000)
logreg_pol.fit(X_train, y_train)

preds = logreg_pol.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98      6144
           1       0.98      0.99      0.98      5817

    accuracy                           0.98     11961
   macro avg       0.98      0.98      0.98     11961
weighted avg       0.98      0.98      0.98     11961

